In [44]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager
from scipy import optimize
from scipy import linalg
import scipy.optimize as so
from scipy.integrate import solve_ivp
import pandas as pd

<h6>We have a coarse grained representation of the population. For our analysis, we assume that the coarse grained representation available to us is a 2x2 next generation matrix. Further, we assume that the highest resolution description of the population is a 3x3 next generation matrix. We calculate the herd immunity threshold curve for 2x2 system -- let us call the points on this curve, the low-resolution herd immunity thresholds (HIT). Then, we investigate what happens if the 3x3 system achieves one of the low-resolution HIT -- does that translate into a high-resolution herd immunity as well?

In [45]:
##Defining the coarse grained system:
n = np.array([0.6, 0.4])
no_g = len(n)
gamma = 1 #Recovery rate
A = np.ones(no_g)
#b = np.random.uniform(low = 0, high = 4, size = (no_g, no_g)) #Transmission parameters i.e. beta_ij for groups i and j
#b = np.array([[3.5, 2, 1.6], [2, 4, 4], [1.5, 2, 0.8]])
b = np.array([[2, 3.5], [2.5, 1.5]]) #Selected arbitrary transmission matrix
NGM = ((b.T * n)/gamma).T
print('NGM = \n', NGM, '\n')


NGM = 
 [[1.2 2.1]
 [1.  0.6]] 



In [46]:
#Computing R0 and dominant eigenvector
Evalues, Evecs = np.linalg.eig(NGM)
R0 = np.max(Evalues)
V = Evecs[:, np.argmax(Evalues)]/np.sum(Evecs[:, np.argmax(Evalues)])
print('R0 and dominant E vector of the 2x2 NGM are: ', R0, V, '\n')

R0 and dominant E vector of the 2x2 NGM are:  2.3798648586948747 [0.64027028 0.35972972] 



In [47]:
def EQN(X, G11_cg, G12_cg, G21_cg, G22_cg, v1_cg, v2_cg, R0): #X = [tg, tv, G12, G13, G22, G23, G32, G33]
    tg, tv, G12, G13, G22, G23, G32, G33 = X
    return [G12*tv + G13*(1-tv) - G12_cg, (G22 + G32)*tv + (G23 + G33)*(1-tv) - G22_cg, v2_cg*(G12*tv + G13*(1-tv)) + v1_cg*(G11_cg - R0), v1_cg*G21_cg*tg + v2_cg*((G22-R0)*tv + G23*(1-tv)), v1_cg*G21_cg*(1-tg) + v2_cg*(G32*tv + (1-tv)*(G33 - R0)), 0, 0, 0]

# while True:
#     soln = so.root(EQN, np.random.rand(8), args = (NGM[0, 0], NGM[0, 1], NGM[1, 0], NGM[1, 1], V[0], V[1], R0))
#     tg, tv, G12_cg, G13_cg, G22_cg, G23_cg, G32_cg, G33_cg = soln.x
#     NGM_hr = np.array([[NGM[0, 0], G12_cg, G13_cg], [tg*NGM[1, 0], G22_cg, G23_cg], [(1-tg)*NGM[1, 0], G32_cg, G33_cg]])
#     if np.all(NGM_hr>=0) and soln.success == True:
#         break

# print(soln.success)
# print('A possible high resolution NG matrix is:', NGM_hr)

Selecting an arbitrary NGM from the set of solutions
NGM_hr = np.array([[1.2, 2.85, 1.98], [.07, 1.44, 0.002], [.93, 1.35, .26]])

print('High resolution NGM: ', NGM_hr)
print('E values of the HR matrix', np.linalg.eig(NGM_hr)[0], '\n')
print('E vectors of the HR matrix', np.linalg.eig(NGM_hr)[1], '\n')

#Define group sizes and b matrix for full system
n_hr = np.array([n[0], 0.5*n[1], 0.5*n[1]]) ##Assuming that the second group is equally divided in the high resolution system

###############################################
#b_hr = (NGM_hr*gamma/n_hr).T          ##  Incorrect calculation for b_hr
###############################################

b_hr = (NGM_hr.T*gamma/n_hr).T
print('b new:', b_hr)
print('NGM calculated again: ', (b_hr.T*n_hr/gamma).T)

True
A possible high resolution NG matrix is: [[1.2        7.75116564 0.25714075]
 [0.17030103 0.91672267 0.07517567]
 [0.82969897 0.34296491 0.30969856]]
High resolution NGM:  [[1.2        7.75116564 0.25714075]
 [0.17030103 0.91672267 0.07517567]
 [0.82969897 0.34296491 0.30969856]]
E values of the HR matrix [2.37986486+0.j         0.02327819+0.29173605j 0.02327819-0.29173605j] 

E vectors of the HR matrix [[ 0.913407  +0.j          0.30814027-0.33790701j  0.30814027+0.33790701j]
 [ 0.12619854+0.j         -0.00463339+0.06289611j -0.00463339-0.06289611j]
 [ 0.38699042+0.j         -0.88706879+0.j         -0.88706879-0.j        ]] 

b new: [[ 2.         12.9186094   0.42856791]
 [ 0.85150513  4.58361336  0.37587834]
 [ 4.14849487  1.71482454  1.54849281]]
NGM calculated again:  [[1.2        7.75116564 0.25714075]
 [0.17030103 0.91672267 0.07517567]
 [0.82969897 0.34296491 0.30969856]]


In [48]:
#Solve HIT for 2x2 system
def growth_rate_eqn(s1, s2, b11, b12, b21, b22, g):
    return 0.5*(b11*s1 + b22*s2 - 2*g + np.sqrt((2*g - b11*s1 - b22*s2)**2 - 4*((b11*b22 - b12*b21)*s1*s2 - g*b11*s1 - g*b22*s2 + g**2)))

#Computes growth rate for any system 
def growth_rate(s, b, gamma): #This function returns the growth rate
    B = (b.T * s).T 
    M = B - np.eye(len(s))*gamma #The largest eigenvalue of M is the growth rate of the epidemic
    EV = linalg.eigvals(M)
    gr = np.amax(EV) #Growth rate
    return gr 

def FS_two_waves(x, b, r, n, g):
    return [(n[k]-x[k])/(n[k] - r[k]) - np.exp(-np.sum(b[k]*(x-r))/g) for k in range(len(n))]

def c_typeII_eqn(c, b, r, n):
    #return np.array([np.sum(b[0]*(1-c[0])*(1-c)*r) + np.log(1-r[0]/n[0]), np.sum(b[1]*(1-c[1])*(1-c)*r) + np.log(1-r[1]/n[1]), np.sum(b[2]*(1-c[2])*(1-c)*r) + np.log(1-r[2]/n[2])])
    return np.array([np.sum(b[k]*(1-c[k])*(1-c)*r) + np.log(1-r[k]/n[k]) for k in range(len(n))])
    
def c_typeII(r, b, n):
    sol = so.root(c_typeII_eqn, 0.8*np.ones(len(n)), args = (b, r, n)) #Using a numerical solver to find the c values
    return (sol.x, sol.success)

In [49]:
#Scanning the variable space of susceptibles, to compute the herd immunity threshold
S2 = np.arange(0.01, n[1]+0.01, 0.05)
S1 = np.array([so.root(growth_rate_eqn, np.random.rand(), args = (s2, b[0, 0], b[0, 1], b[1, 0], b[1, 1], gamma)).x[0] for s2 in S2])
#S1 = np.array(S1)
S1_plot, S2_plot = [], []
growth_rates_max = []
growth_rates_min = []
FS_max = []
FS_min = []


a_list = np.linspace(0, 1, 20)
#a_list = [0.5]
plot_data = []
aux_data = []

max_guesses = 10

print('The susceptible population of the 2nd group in 2x2 system has to be distributed among the the 2nd and 3rd groups in the 3x3 system', '\n')
print('a*100 % are placed in the new second group, and (1-a)*100 % are placed in the new third group', '\n')

for a in a_list:
    for k in range(0, len(S1)):
    #print(k)
        s1_req, s2_req = S1[k], S2[k]
        #Bounds for the parameter a, such that s<n
        bound_1 = 1-n_hr[2]/s2_req 
        bound_2 = n_hr[1]/s2_req
        if (bound_1 < bound_2 and a>bound_1 and a<bound_2) or (bound_2 > bound_1 and a>bound_2 and a<bound_1):
            s_req_hr = np.array([s1_req, a*s2_req, (1-a)*s2_req])
            
            gr = np.real(growth_rate(s_req_hr, b_hr, gamma))
            
            guess = np.array([np.random.uniform(n_hr[k] - s_req_hr[k], n_hr[k]) for k in range(len(n_hr))])
            if gr>0:
                # for i in range(max_guesses):
                #     root = so.root(FS_two_waves, n_hr*np.random.rand(len(n_hr)), args = (b_hr, n_hr - s_req_hr, n_hr, gamma))
                #     if root.success == True and np.all(root.x - (n_hr-s_req_hr) > 0) and np.all(root.x < n_hr):
                #         break
                root = so.root(FS_two_waves, guess, args = (b_hr, n_hr - s_req_hr, n_hr, gamma))
                if root.success:
                    plot_data.append([s1_req, s2_req, a, gr, np.sum(root.x)])
                    tmp = root.x - (n_hr - s_req_hr)
                    aux_data.append([s1_req, s2_req, a, gr, tmp[0], tmp[1], tmp[2], root.x[0], root.x[1], root.x[2]])
                    
            # while True:
            #     root = so.root(FS_two_waves, n_hr*np.random.rand(len(n_hr)), args = (b_hr, n_hr - s_req_hr, n_hr, gamma))
            #     if root.success == True and np.all(root.x - n_hr - s_req_hr>-1e-4): ##There is a mistake in this line
            #         break
            # if root.success == True:# and np.all(root.x - (n_hr-s_req_hr) > -1e-4):        
            #     plot_data.append([s1_req, s2_req, a, gr, np.sum(root.x)])
            #     tmp = root.x - (n_hr - s_req_hr)
            #     aux_data.append([s1_req, s2_req, a, gr, tmp[0], tmp[1], tmp[2]])
                




The susceptible population of the 2nd group in 2x2 system has to be distributed among the the 2nd and 3rd groups in the 3x3 system 

a*100 % are placed in the new second group, and (1-a)*100 % are placed in the new third group 



C:\Users\PKOLLE~1\AppData\Local\Temp/ipykernel_5336/2487180645.py:14: RuntimeWarning: invalid value encountered in double_scalars
  return [(n[k]-x[k])/(n[k] - r[k]) - np.exp(-np.sum(b[k]*(x-r))/g) for k in range(len(n))]
C:\Users\PKOLLE~1\AppData\Local\Temp/ipykernel_5336/2487180645.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  return [(n[k]-x[k])/(n[k] - r[k]) - np.exp(-np.sum(b[k]*(x-r))/g) for k in range(len(n))]


In [50]:
data = pd.DataFrame(plot_data, columns = [r's1', r's2', r'a', r'growth rate', r'final size'])
aux_df = pd.DataFrame(aux_data, columns = [r's1', r's2', r'a', r'growth rate', r'fs-hit 0',  r'fs-hit 1',  r'fs-hit 2', 'fs 0', 'fs 1', 'fs 2'])

In [51]:
aux_df.sample(5)

,s1,s2,a,growth rate,fs-hit 0,fs-hit 1,fs-hit 2,fs 0,fs 1,fs 2
44,0.317189,0.11,0.894737,0.145093,0.099374,0.013572,0.004136,0.382185,0.115151,0.192557
5,0.113022,0.36,0.473684,0.149294,0.045188,0.030798,0.051996,0.532166,0.060271,0.062523
45,0.260274,0.16,0.894737,0.244000,0.131703,0.033945,0.007751,0.471429,0.090787,0.190909
21,0.388060,0.06,0.684211,0.010473,0.009005,0.000399,0.000728,0.220945,0.159347,0.181780
25,0.174535,0.26,0.684211,0.260619,0.099975,0.049202,0.034883,0.525440,0.071307,0.152778


In [52]:
%matplotlib widget

In [53]:
aux_df.plot(x = 'growth rate', y = ['fs-hit 0', 'fs-hit 1', 'fs-hit 2'], kind = 'line', subplots = True, marker = '.', lw = 0, markersize = '5', alpha = 0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:xlabel='growth rate'>,
       <AxesSubplot:xlabel='growth rate'>,
       <AxesSubplot:xlabel='growth rate'>], dtype=object)

In [54]:
aux_df.plot(x = 'fs-hit 0', y = 'fs-hit 2', kind = 'scatter', alpha = 0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='fs-hit 0', ylabel='fs-hit 2'>

In [55]:
data['expected final size'] = 1 - data['s1'] - data['s2']


In [56]:
data.sample(20)

,s1,s2,a,growth rate,final size,expected final size
17,0.260274,0.16,0.631579,0.076652,0.647798,0.579726
3,0.174535,0.26,0.473684,0.052187,0.615324,0.565465
33,0.388060,0.06,0.789474,0.037161,0.586655,0.551940
47,0.478736,0.01,0.947368,0.009605,0.520710,0.511264
25,0.174535,0.26,0.684211,0.260619,0.749524,0.565465
7,0.213562,0.21,0.526316,0.052851,0.625826,0.576438
24,0.213562,0.21,0.684211,0.180543,0.716515,0.576438
29,0.260274,0.16,0.736842,0.143066,0.695520,0.579726
45,0.260274,0.16,0.894737,0.244000,0.753125,0.579726
28,0.317189,0.11,0.736842,0.074682,0.638906,0.572811


In [57]:
fig = plt.figure()
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.scatter(data['s1'], data['s2'], data['growth rate'], s = 0.5)

ax.set_xlabel(r'$s_1$')
ax.set_ylabel(r'$s_2$')
ax.set_zlabel('Effective \n growth rate')
#ax.set_xlim(0, n[0])
#ax.set_ylim(0, n[1])

ax = fig.add_subplot(1, 2, 2, projection='3d')
#ax = fig.add_subplot(projection='3d')
ax.scatter(data['s1'], data['s2'], data['final size'], s = 0.5, alpha = 0.5)

ax.plot3D(S1, S2, 1-S1-S2, 'gray')

ax.set_xlabel(r'$s_1$')
ax.set_ylabel(r'$s_2$')
ax.set_zlabel(r'Final size')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Final size')

In [26]:
def SIR_equations(t, X, b, g, no_g): ##Define the dynamical equations for SIR model with 3 groups and arbitrary transmission
    S = X[0:no_g]
    I = X[no_g:2*no_g]
    R = X[2*no_g:3*no_g]

    dsdt = [(-S[k]*(np.sum(b[k]*I))) for k in range(no_g)]
    didt = [(S[k]*(np.sum(b[k]*I)) - g*I[k]) for k in range(no_g)]
    drdt = [g*I[k] for k in range(no_g)]
    return np.concatenate((dsdt, didt, drdt))

In [ ]:
c_typeII()

In [ ]:
eps_i = 0.005 ## Threshold for i(t) at which intervention is imposed
eps_f = 0.5*1e-3 ## Threshold for i(t) below which intervention is released
ti = 0
tf = 80
dt = 0.05
T = np.arange(ti, tf, dt)
s_init, i_init, r_init = n - 1/3*1e-3, np.ones(no_g)*1/3*1e-3, np.zeros(no_g)
X_init = np.concatenate((s_init, i_init, r_init))
# 1/3*1e-3 is i(0) for each of the groups.

#Scanning the variable space of susceptibles, to compute the herd immunity threshold
S2 = np.arange(0.01, n[1]+0.01, 0.01)
S1 = np.array([so.root(growth_rate_eqn, np.random.rand(), args = (s2, b[0, 0], b[0, 1], b[1, 0], b[1, 1], gamma)).x[0] for s2 in S2])
#S1 = np.array(S1)
S1_plot, S2_plot = [], []
growth_rates_max = []
growth_rates_min = []
FS_max = []
FS_min = []

FS = []
growth_rates = []
print('The susceptible population of the 2nd group in 2x2 system has to be distributed among the the 2nd and 3rd groups in the 3x3 system', '\n')
print('a% are placed in the new second group, and (1-a)% are placed in the new third group', '\n')
for k in range(0, len(S1)):
    #print(k)
    s1_req, s2_req = S1[k], S2[k]
    #s_req_hr = np.array([s1_req, s2_req/n[1]*n_hr[1], s2_req/n[1]*n_hr[2]])
    #Bounds for the parameter a, such that s<n
    bound_1 = 1-n_hr[2]/s2_req 
    bound_2 = n_hr[1]/s2_req
    #print('bounds', bound_1, bound_2)
    if bound_1 < bound_2:
        a_list = np.linspace(max(bound_1, 0), min(bound_2, 1), 10)
        #a = np.random.uniform(max(bound_1, 0), min(bound_2, 1))
    else:
        a_list = np.linspace(max(bound_2, 0), min(bound_1, 1), 10)
        #a = np.random.uniform(max(bound_2, 0), min(bound_1, 1))
    

    if np.any(a_list<0) or np.any(a_list>1):
        print('Fault')
        break
    
    S1_plot.append(s1_req)
    S2_plot.append(s2_req)

    for a in a_list:
        s_req_hr = np.array([s1_req, a*s2_req, (1-a)*s2_req])
        growth_rates.append(np.real(growth_rate(s_req_hr, b_hr, gamma)))
        root = so.root(FS_two_waves, n_hr*np.random.rand(len(n_hr)), args = (b_hr, n_hr - s_req_hr, n_hr, gamma))
        if np.all(root.x)>-1e-4:
            FS.append(np.sum(root.x))
    growth_rates_max.append(np.max(growth_rates))
    growth_rates_min.append(np.min(growth_rates))
    FS_max.append(np.max(FS))
    FS_min.append(np.min(FS))
    
    # if k == len(S1)-2:
    #     print('s^f = ', s_req_hr, '\n')
    #     print('Growth Rate =', growth_rate(s_req_hr, b_hr, gamma), '\n')

S1_plot = np.array(S1_plot)
S2_plot = np.array(S2_plot)



In [61]:
%matplotlib widget

In [67]:
fig = plt.figure()
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot3D(S1_plot, S2_plot, growth_rates_min, 'gray')
ax.plot3D(S1_plot, S2_plot, growth_rates_max, 'black')

xx, yy = np.meshgrid(S1_plot, S2_plot)
zz = np.zeros(xx.shape)
ax.plot_surface(xx, yy, zz, alpha = 0.5)

# im = ax.scatter(S1_plot[tmp], S2_plot[tmp], c = np.real(growth_rates[tmp]), cmap = 'Greys', alpha = 0.8)
# fig.colorbar(im, ax=ax, location = 'bottom')

#ax.set_xlim3d(s0[2], )
#ax.set_ylim3d(s1[2], )
ax.set_xlabel(r'$s_1$')
ax.set_ylabel(r'$s_2$')
ax.set_zlabel('Effective \n growth rate')
#ax.set_xlim(0, n[0])
#ax.set_ylim(0, n[1])

ax = fig.add_subplot(1, 2, 2, projection='3d')

ax.plot3D(S1_plot, S2_plot, FS_min, 'gray')
ax.plot3D(S1_plot, S2_plot, FS_max, 'black')
ax.plot3D(S1_plot, S2_plot, 1-S1_plot-S2_plot, 'red')
ax.set_xlabel(r'$s_1$')
ax.set_ylabel(r'$s_2$')
ax.set_zlabel(r'Final size')
# for t in np.linspace(0, 1, 20):
#     n_hr = [n[0], t*n[1], (1-t)*n[1]]
#     b_hr = (NGM_hr*gamma/n_hr).T


# t = np.random.rand()
# G21, G31 = t*NGM[1, 0], (1-t)*NGM[1, 0]
# v2, t = np.random.rand(), np.random.rand()
# v3 = t*v2
# v1 = 1 - v2*(1+t)

# print(v1, v2, v3, G21, G31)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Final size')

In [68]:
ax.plot3D(S1_plot, S2_plot, 1-S1_plot-S2_plot, 'green', ls = '--')

In [66]:
plt.show()

n1 n2

n_hr 1, 2, 3
n1 = n_hr 1
n_hr 2
n-Hr 


s1 s2
s1 hr

s2


In [63]:
c_hr = c_typeII(n_hr-s_req_hr, b_hr, n_hr)
print(c_hr)

(array([-0.3558863,  1.       ,  1.       ]), True)


#Scanning the variable space of susceptibles, to compute the herd immunity threshold
S2 = np.arange(0.01, n[1]+0.01, 0.01)
S1 = np.array([so.root(growth_rate_eqn, np.random.rand(), args = (s2, b[0, 0], b[0, 1], b[1, 0], b[1, 1], gamma)).x[0] for s2 in S2])
#S1 = np.array(S1)
S1_plot, S2_plot = [], []
growth_rates_max = []
growth_rates_min = []

print('The susceptible population of the 2nd group in 2x2 system has to be distributed among the the 2nd and 3rd groups in the 3x3 system', '\n')
print('a% are placed in the new second group, and (1-a)% are placed in the new third group', '\n')
for k in range(0, len(S1)):
    print(k)
    s1_req, s2_req = S1[k], S2[k]
    #s_req_hr = np.array([s1_req, s2_req/n[1]*n_hr[1], s2_req/n[1]*n_hr[2]])
    #Bounds for the parameter a, such that s<n
    bound_1 = 1-n_hr[2]/s2_req 
    bound_2 = n_hr[1]/s2_req
    print('bounds', bound_1, bound_2)
    if bound_1 < bound_2:
        a_list = np.linspace(max(bound_1, 0), min(bound_2, 1), 10)
        #a = np.random.uniform(max(bound_1, 0), min(bound_2, 1))
    else:
        a_list = np.linspace(max(bound_2, 0), min(bound_1, 1), 10)
        #a = np.random.uniform(max(bound_2, 0), min(bound_1, 1))
    
    if np.any(a_list<0) or np.any(a_list>1):
        print('Fault')
        break
    
    S1_plot.append(s1_req)
    S2_plot.append(s2_req)
    growth_rates = []
    for a in a_list:
        s_req_hr = np.array([s1_req, a*s2_req, (1-a)*s2_req])
        growth_rates.append(np.real(growth_rate(s_req_hr, b_hr, gamma)))

    growth_rates_max.append(np.max(growth_rates))
    growth_rates_min.append(np.min(growth_rates))
    
    # if k == len(S1)-2:
    #     print('s^f = ', s_req_hr, '\n')
    #     print('Growth Rate =', growth_rate(s_req_hr, b_hr, gamma), '\n')

#print('s req full', s_req_hr, '   s1_req and s2_req', s1_req, s2_req)
S1_plot = np.array(S1_plot)
S2_plot = np.array(S2_plot)

growth_rates = np.array(growth_rates)
tmp = growth_rates>0


fig, ax = plt.subplots(1, 1)
ax.plot(S1, S2)

im = ax.scatter(S1_plot[tmp], S2_plot[tmp], c = np.real(growth_rates[tmp]), cmap = 'Greys', alpha = 0.8)
fig.colorbar(im, ax=ax, location = 'bottom')

ax.set_xlabel(r'$s_1$')
ax.set_ylabel(r'$s_2$')
#ax.set_xlim(0, n[0])
#ax.set_ylim(0, n[1])
plt.show()

# for t in np.linspace(0, 1, 20):
#     n_hr = [n[0], t*n[1], (1-t)*n[1]]
#     b_hr = (NGM_hr*gamma/n_hr).T


# t = np.random.rand()
# G21, G31 = t*NGM[1, 0], (1-t)*NGM[1, 0]
# v2, t = np.random.rand(), np.random.rand()
# v3 = t*v2
# v1 = 1 - v2*(1+t)

# print(v1, v2, v3, G21, G31)

# #We have three more variables to solve: G22, G23, G32, G33. But only three equations are available. So we choose G22 randomly
# G22 = R0*np.random.rand() #Can lie between 0 and R0 -- think about this
# print(G22, '\n \n')

# ## Solves for G12 and G13
# def EQN1(X, v2, v3, R0, G12_cg, G11):
#     return np.array([X[0]*v2 + X[1]*v3 - G12_cg*(v2+v3), (X[0] - G11 + R0)*v2 + (X[1] - G11 + R0)*v3 + G11 - R0])

# print('G12 and G13: \n', so.root(EQN1, np.random.rand(2), args = (v2, v3, R0, NGM[0, 1], G11)))



# ## Solves for X = [G23, G32, G33]
# def EQN2(X, v2, v3, R0, G21, G22, G31, G22_cg):
#     return np.array([(G22 - G21 - R0)*v2 + (X[0] - G21)*v3 + G21, (X[1]-G31)*v2 + (X[2]-G31-R0)*v3 + G31, (G22+X[1])*v2 + (X[2]+X[0])*v3 - G22_cg*(v2+v3)])

# print('\n G23, G32 and G33: \n', so.root(EQN2, np.random.rand(3), args = (v2, v3, R0, G21, G22, G31, NGM[1, 1])))

#constraint21 = optimize.NonlinearConstraint(lambda x: x[0]+x[1] - NGM[1, 0], lb=0, ub=0)
#constraint_positive = optimize.NonlinearConstraint(lambda x: x, lb=0, ub=np.inf)
#constraint21 = optimize.NonlinearConstraint(lambda x: x[0]+x[1] - NGM[1, 0], lb=0, ub=0)

##############################################################################################
# NGM_cg = np.zeros((2, 2))
# NGM_cg[0, 0], NGM_cg[0, 1], NGM_cg[1, 0] = NGM[0, 0], np.sum(NGM[0, 1:]*V[1:])/np.sum(V[1:]), np.sum(NGM[1:, 0])
# NGM_cg[1, 1] = np.sum([NGM[k, l]*V[l] for k in range(1, no_g) for l in range(1, no_g)])/np.sum(V[1:])

# Evalues, Evecs = np.linalg.eig(NGM)
# R0_cg = np.max(Evalues)

# n_cg = np.array([n[0], np.sum(n[1:])])
# no_g_cg = len(n_cg)
# b_cg = (NGM_cg*gamma/n_cg).T
# A_cg = np.ones(no_g_cg)
# NGM_cg, n_cg, b_cg
# #V = Evecs[:, np.argmax(Evalues)]/np.sum(Evecs[:, np.argmax(Evalues)])

# root_ord = so.root(FS, 0.9*n, args = (b, n, gamma)) #Using a numerical solver to find the final size
# FS_ord = root_ord.x #The final sizes without intervention